In [1]:
import pandas as pd
import numpy as np

pd.set_option('precision', 13)
pd.options.display.max_rows = 100

In [2]:
# загружаем города и поселки

work_cities = pd.read_csv('C:/00_Projects/Russia_color/all_cities_original.csv')

### Проверяем пустые значения ###

In [3]:
# смотрим пустые значения

work_cities[work_cities.offname.isnull()]

,aolevel,formalname,offname,shortname,plaincode,regioncode,areacode,citycode,okato,oktmo,region,mun_code,mun_district


In [4]:
# в столбце city (бывшем formalname)

work_cities[work_cities.formalname.isnull()]

,aolevel,formalname,offname,shortname,plaincode,regioncode,areacode,citycode,okato,oktmo,region,mun_code,mun_district


### Переименовываем столбцы ###

In [5]:
work_cities.columns

Index(['aolevel', 'formalname', 'offname', 'shortname', 'plaincode',
       'regioncode', 'areacode', 'citycode', 'okato', 'oktmo', 'region',
       'mun_code', 'mun_district'],
      dtype='object')

In [6]:
# оставляю нужные столбцы (беру formalname, в offname есть опечатки в названиях)

work_cities = work_cities[['aolevel', 'shortname', 'formalname',
                          'mun_district', 'region', 'okato', 'oktmo']]

In [7]:
# перименовываю столбцы (city у меня - любой населенный пункт)

work_cities = work_cities.rename(columns={
    'shortname': 'city_type',
    'formalname': 'city'})

## Удаляем дубликаты ##

Хотя мы удаляли дубликаты на предыдущем этапе, они все равно есть.

Но на всякий случай проверим, точно ли это идентичные записи.

**P.S.** У меня есть гипотеза, что это записи из разных таблиц. Т.е. некоторые записи дублируют в другой регион по ошибке или населенный пункт меняет принадлежность, но остается в таблице бывшего родительского региона с новым кодом.

### Проверяем ###

In [8]:
# заводим тестовый датафрейм

test = work_cities[work_cities.duplicated(keep=False)]

In [9]:
# смотрим индексы совпадающих строк

print(test.groupby(test.columns.tolist()).apply(lambda x: x.index.tolist()).values.tolist())

[[160481, 162429], [159581, 160173], [144845, 144865], [139446, 140514], [157537, 158793], [69133, 69721], [160619, 163217], [68518, 68660], [158522, 161012], [68394, 69271], [3446, 5326], [160385, 161038], [65424, 67812], [85139, 85541], [83996, 85446], [159536, 162657], [39370, 39464], [164299, 164548], [98729, 99086], [36226, 36640], [2831, 5324], [250, 1950], [1260, 5195]]


In [10]:
# нужно вставить индексы из прошлой ячейки

work_cities.loc[[160481, 162429]]

,aolevel,city_type,city,mun_district,region,okato,oktmo
160481,6,д,Горки,Ростовский,Ярославская область,78237820009.0,78637441276.0
162429,6,д,Горки,Ростовский,Ярославская область,78237820009.0,78637441276.0


In [11]:
work_cities.loc[[2831, 5324]]

,aolevel,city_type,city,mun_district,region,okato,oktmo
2831,6,с,Леуза,Кигинский,Башкортостан,80236840001.0,80636440101.0
5324,6,с,Леуза,Кигинский,Башкортостан,80236840001.0,80636440101.0


### Удаляем ###

In [12]:
work_cities.shape

(166093, 7)

In [13]:
work_cities = work_cities.drop_duplicates()

In [14]:
work_cities.shape

(166070, 7)

**Удаляем районы (муниципальные образования)**

Они остались с прошлого шага

In [15]:
# aolevel - 3 и city_type - р-н

work_cities = work_cities[~((work_cities.aolevel == 3) &
           (work_cities.city_type == 'р-н'))]

In [16]:
work_cities.shape

(164269, 7)

In [17]:
work_cities

,aolevel,city_type,city,mun_district,region,okato,oktmo
0,6,х,Советский,Майкопский,Адыгея,79222000044.0,79622420136.0
1,6,ст,Шенджий,Тахтамукайский,Адыгея,79230000025.0,79630419101.0
2,6,ст-ца,Ханская,Майкоп,Адыгея,79401000008.0,79701000141.0
3,6,аул,Гатлукай,Адыгейск,Адыгея,79403000001.0,79703000106.0
4,6,х,Псекупс,Адыгейск,Адыгея,79403000002.0,79703000111.0
...,...,...,...,...,...,...,...
166088,6,с,Оборонное,Оборонное,Севастополь,67263000000.0,67302000146.0
166089,6,с,Морозовка,Морозовка,Севастополь,67263000000.0,67300000.0
166090,6,с,Родниковое,Родниковое,Севастополь,67263000000.0,67306000141.0
166091,6,п,Кача,Кача,Севастополь,67269000000.0,67320000051.0


**Меняем "г." на "г", "пгт." на "пгт" и "сп." на "сп"**

In [18]:
work_cities.city_type.replace('г.', 'г', inplace=True)
work_cities.city_type.replace('сп.', 'сп', inplace=True)
work_cities.city_type.replace('пгт.', 'пгт', inplace=True)

In [19]:
work_cities[work_cities.city_type == 'г.'].shape 

(0, 7)

### Записываем в файл ###


In [20]:
work_cities.to_csv('C:/00_Projects/Russia_color/all_cities.csv',
                  sep=',', encoding='utf-8', index=False)

## Чистим города и поселки ##

In [21]:
work_cities.sample(10)

,aolevel,city_type,city,mun_district,region,okato,oktmo
24703,6,с,Майорка,Чарышский,Алтайский край,1258811003.0,1658411111.0
160973,6,д,Бутакино,Рыбинский,Ярославская область,78240810008.0,78640410151.0
117565,6,д,Лехово,Новоржевский,Псковская область,58223805015.0,58623405226.0
48965,6,д,Котельное,Никольский,Вологодская область,19234812006.0,19634456246.0
133807,6,д,Чальцево,Краснинский,Смоленская область,66224810014.0,66624455187.0
23316,6,тер,сдт Барнаульское,Бийский,Алтайский край,1204859001.0,1604459101.0
103331,6,д,Калинина,Болховский,Орловская область,54204834008.0,54604434136.0
78131,6,д,Журавлево,Бокситогорский,Ленинградская область,41203000124.0,41603155124.0
142029,6,д,Задорье,Жарковский,Тверская область,28214832004.0,28614440192.0
160841,6,д,Юрино,Переславский,Ярославская область,78232816039.0,78705000561.0


In [22]:
# смотрим сколько вообще  типов населенных пунктов есть в базе

work_cities.city_type.nunique()

61

In [23]:
# и каких

(work_cities
.groupby('city_type', as_index=False)
.agg({'aolevel': 'count'})
.rename(columns={'aolevel': 'city_number'})
.sort_values('city_type', ascending=False)
)

,city_type,city_number
60,х,5777
59,у,222
58,тер,2275
57,ст-ца,444
56,ст,516
55,сп,51
54,снт,1678
53,сл,89
52,с/с,751
51,с/п,733


Удаляем то, что имеет адрес, но не является населенным пунктом: ж/д будки, автодороги, а также промзоны и территории.

In [24]:
# выгружаем все типы

print(work_cities.city_type.value_counts().index.to_list())

['д', 'с', 'п', 'х', 'тер', 'снт', 'г', 'с/с', 'с/п', 'пгт', 'рп', 'ст', 'нп', 'ст-ца', 'ж/д_ст', 'дп', 'рзд', 'п/ст', 'у', 'аул', 'ж/д_рзд', 'м', 'массив', 'починок', 'мкр', 'сл', 'п. ж/д ст.', 'ж/д_будка', 'автодорога', 'заимка', 'высел', 'аал', 'сп', 'промзона', 'гп', 'ж/д_казарм', 'ж/д о.п.', 'ж/д_оп', 'г-к', 'кв-л', 'ж/д_платф', 'арбан', 'казарма', 'жилрайон', 'кордон', 'остров', 'кп', 'с/мо', 'ж/д_пост', 'п/о', 'ж/д п.п.', 'лпх', 'пл.р-н', 'пос.рзд', 'погост', 'с/а', 'п. ст.', 'ж/д пл-ка', 'ж/д б-ка', 'с/о', 'ж/д бл-ст']


In [25]:
# отбираем ненужные

no_city = ['автодорога',
           'ж/д б-ка', 'ж/д пл-ка', 'ж/д_будка',
           'ж/д_казарм', 'ж/д_оп', 'ж/д_платф', 'ж/д_пост', 'ж/д_рзд',
           'казарма', 'п/о',
           'промзона', 'тер']

In [26]:
# Удаляем ненужные записи

work_cities = work_cities.loc[~work_cities.city_type.isin(no_city)]

In [27]:
work_cities.shape

(161536, 7)

### Записываем в файл ###

In [ ]:
work_cities.to_csv('C:/00_Projects/Russia_color/all_cities_clean.csv',
                  sep=',', encoding='utf-8', index=False)